# Loading Spatial Data Formats

For more comprehensive coverage of both basic and advanced SQL, please refer to the course textbook:
https://github.com/giswqs/geog-414



In [ ]:
# %pip install duckdb leafmap

### Library Import and Initial Setup

In [ ]:
import duckdb
import leafmap
import pandas as pd

## Installing and Loading Extensions

In [ ]:
con = duckdb.connect()

In [ ]:
con.install_extension("httpfs")
con.load_extension("httpfs")

In [ ]:
con.install_extension("spatial")
con.load_extension("spatial")

## Downloading Sample Data

In [ ]:
url = "https://data.gishub.org/duckdb/cities.zip"
leafmap.download_file(url, unzip=True)

## Loading CSV Files with Coordinates

### Basic CSV Loading with Automatic Detection

In [ ]:
con.read_csv("cities.csv")

### Overriding Automatic Detection

In [ ]:
con.read_csv("cities.csv", header=True, sep=",")

### Parallel CSV Reading for Large Files

In [ ]:
con.read_csv("cities.csv", parallel=True)

### Querying CSV Files Directly in SQL

In [ ]:
con.sql("SELECT * FROM 'cities.csv'")

In [ ]:
con.sql("SELECT * FROM read_csv_auto('cities.csv')")

### Performance Considerations and Best Practices

In [ ]:
con.sql("COPY (SELECT * FROM 'cities.csv') TO 'cities-demo.parquet'")

In [ ]:
con.sql("DESCRIBE SELECT * FROM 'cities.csv'").show()

In [ ]:
con.sql("SELECT * FROM 'cities.csv' LIMIT 100").show()

## Loading JSON Files

### Reading JSON with Automatic Schema Detection

In [ ]:
con.read_json("cities.json")

In [ ]:
con.sql("SELECT * FROM 'cities.json'")

In [ ]:
con.sql("SELECT * FROM read_json_auto('cities.json')")

## Querying Pandas DataFrames Directly

### Loading Data into Pandas and Querying with SQL

In [ ]:
df = pd.read_csv("cities.csv")
df

In [ ]:
con.sql("SELECT * FROM df").fetchall()

## Loading Parquet Files for Performance

### Reading Parquet Files in Python

In [ ]:
con.read_parquet("cities.parquet")

### Querying Parquet Files Directly in SQL

In [ ]:
con.sql("SELECT * FROM 'cities.parquet'")

In [ ]:
con.sql("SELECT * FROM read_parquet('cities.parquet')")

### Cloud Storage and Remote Parquet Files

In [ ]:
con.sql("SELECT * FROM 'https://data.gishub.org/duckdb/cities.parquet'")

In [ ]:
url = "s3://us-west-2.opendata.source.coop/vida/google-microsoft-osm-open-buildings/geoparquet/by_country/country_iso=USA/USA.parquet"
con.sql(f"SELECT * FROM '{url}' LIMIT 10")

### When to Convert to Parquet

In [ ]:
con.sql("COPY (SELECT * FROM 'cities.csv') TO 'cities-demo.parquet'")

## Loading GeoJSON Files with Spatial Geometries

### Discovering Available Spatial Formats

In [ ]:
con.sql("SELECT * FROM ST_Drivers()").show(max_rows=100)

### Loading GeoJSON with ST_Read()

In [ ]:
con.sql("SELECT * FROM ST_Read('cities.geojson')")

### DuckDB's Shorthand SQL Syntax

In [ ]:
con.sql("FROM ST_Read('cities.geojson')")

### Creating Persistent Spatial Tables

In [ ]:
con.sql("CREATE TABLE cities AS SELECT * FROM ST_Read('cities.geojson')")

### Querying the Spatial Table

In [ ]:
con.table("cities")

In [ ]:
con.sql("SELECT * FROM cities")

## Loading Shapefiles into Modern Workflows

### The Shapefile Multi-File Structure

### Loading Shapefiles with ST_Read()

In [ ]:
con.sql("SELECT * FROM ST_Read('cities.shp')")

In [ ]:
con.sql("FROM ST_Read('cities.shp')")

### Creating Tables from Shapefiles

In [ ]:
con.sql(
    """
        CREATE TABLE IF NOT EXISTS cities2 AS
        SELECT * FROM ST_Read('cities.shp')
    """
)

In [ ]:
con.table("cities2")

In [ ]:
con.sql("SELECT * FROM cities2")

In [ ]:
con.sql("COPY cities2 TO 'cities2.parquet'")

## Loading GeoParquet for Cloud-Native Spatial Analysis

### Reading Local GeoParquet Files

In [ ]:
con.sql("SELECT * FROM 'cities.parquet'")

### Converting WKB Geometries to DuckDB's Spatial Type

In [ ]:
con.sql("FROM 'cities.parquet'")

### Loading GeoParquet from Cloud Storage

In [ ]:
con.sql(
    """
        SELECT *  FROM 's3://us-west-2.opendata.source.coop/fused/overture/2025-06-25-0/theme=divisions/type=division/*.parquet'
    """
)

In [ ]:
con.sql(
    """
        SELECT COUNT(*)  FROM 's3://us-west-2.opendata.source.coop/fused/overture/2025-06-25-0/theme=divisions/type=division/*.parquet'
    """
)

## Data Loading Performance Strategies

In [ ]:
con.sql("FROM 'cities.parquet' USING SAMPLE 10%")

In [ ]:
con.sql("SELECT name, population FROM 'cities.parquet'")

In [ ]:
con.sql("SELECT * FROM 'cities.parquet' WHERE population > 1000000")

In [ ]:
con.sql("SELECT * FROM 'cities.parquet' LIMIT 100")